# StackExchange questions and Tags - Modélisation

This Notebook is about Stackoverflow questions tag prediction 
- I start by importing libraries and Data 
- I make a first unsupervised approch with LDA
- I make different approaches for feature engineering : Bag of Words and TF-IDF
- I finally try supervised approch to classify and predict the tags

## Libraries and frameworks

In [1]:
!pip install tqdm
!pip install flake8 --ignore-installed
!pip install pycodestyle_magic
!pip install wordcloud
!pip install gensim
!pip install pyLDAvis

  Using cached https://files.pythonhosted.org/packages/f8/1f/7ea40d1e4146ea55dbab41cda1376db092a75794914169aabd7e8d7a7def/flake8-3.7.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/0c/04a353e104d2f324f8ee5f4b32012618c1c86dd79e52a433b64fceed511b/pycodestyle-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/c6/44694103f8c221443ee6b0041f69e2740d89a25641e62fb4f2ee568f2f9c/entrypoints-0.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/89/479dc97e18549e21354893e4ee4ef36db1d237534982482c3681ee6e7b57/mccabe-0.6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/f2/ed0ffb887f8138a8fe5a621b8c0bb9598bfb3989e029f6c6a85ee66628ee/pyflakes-2.1.1-py2.py3-none-any.whl


ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [47]:
import pandas as pd
import numpy as np
from pprint import pprint

from tqdm import tqdm

from bs4 import BeautifulSoup
import nltk
import heapq
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import scipy.sparse
import pickle

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn  # don't skip this
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib import pyplot as plt

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s \
: %(levelname)s : %(message)s', level=logging.ERROR)

In [49]:
%load_ext pycodestyle_magic
%pycodestyle_on
%matplotlib inline

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

pyLDAvis.enable_notebook()

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


## Data

In [9]:
df = pd.read_csv('C:/Users/ayoub/OCP6 Data/df.csv', sep=',')

In [19]:
df.head()

,Body,Title,Tags,AnswerCount,FavoriteCount,tokens,title_tokens,tags,ntags,t,sentence,tsentence
0,I want to use a track-bar to change a form's o...,Convert Decimal to Double,<c#><floating-point><type-conversion><double><...,13,50,"['want', 'use', 'track-bar', 'chang', ""form'"",...","['convert', 'decim', 'doubl']","['c#', 'floating-point', 'type-conversion', 'd...",5,"['convert', 'decim', 'doubl', 'want', 'use', '...",convert decim doubl want use track-bar chang f...,c# floating-point type-conversion double decimal
1,I have an absolutely positioned div containing...,Percentage width child element in absolutely p...,<html><css><internet-explorer-7>,6,11,"['absolut', 'posit', 'div', 'contain', 'sever'...","['percentag', 'width', 'child', 'element', 'ab...","['html', 'css', 'internet-explorer-7']",3,"['percentag', 'width', 'child', 'element', 'ab...",percentag width child element absolut_posit pa...,html css internet-explorer-7
2,Given a DateTime representing a person's birth...,How do I calculate someone's age in C#?,<c#><.net><datetime>,61,443,"['given', 'datetim', 'repres', ""person'"", 'bir...","['calcul', ""someone'"", 'age', 'c#?']","['c#', '.net', 'datetime']",3,"['calcul', ""someone'"", 'age', 'c#?', 'given', ...",calcul someone' age c#? given datetim repres p...,c# .net datetime
3,"Given a specific DateTime value, how do I disp...",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,37,540,"['given', 'specif', 'datetim', 'value,', 'disp...","['calcul', 'rel', 'time', 'c#']","['c#', 'datetime', 'time', 'datediff', 'relati...",5,"['calcul', 'rel', 'time', 'c#', 'given', 'spec...","calcul rel time c# given specif datetim value,...",c# datetime time datediff relative-time-span
4,Is there a standard way for a web server to be...,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,24,147,"['standard', 'way', 'web', 'server', 'abl', 'd...","['determin', ""user'"", 'timezon']","['html', 'browser', 'timezone', 'user-agent', ...",5,"['determin', ""user'"", 'timezon', 'standard', '...",determin user' timezon standard way web server...,html browser timezone user-agent timezone-offset
5,What is the difference between Math.Floor() an...,Difference between Math.Floor() and Math.Trunc...,<.net><math>,10,58,"['differ', 'math.floor()', 'math.truncate()', ...","['differ', 'math.floor()', 'math.truncate()']","['.net', 'math']",2,"['differ', 'math.floor()', 'math.truncate()', ...",differ math.floor() math.truncate() differ mat...,.net math
6,How do you expose a LINQ query as an ASMX web ...,Filling a DataSet or DataTable from a LINQ que...,<c#><linq><web-services><.net-3.5>,7,14,"['expos', 'linq', 'queri', 'asmx', 'web', 'ser...","['fill', 'dataset', 'datat', 'linq', 'queri', ...","['c#', 'linq', 'web-services', '.net-3.5']",4,"['fill', 'dataset_datat', 'linq_queri', 'resul...",fill dataset_datat linq_queri result set expos...,c# linq web-services .net-3.5
7,How do I store binary data in MySQL?\n,Binary Data in MySQL,<mysql><database><binary-data><data-storage>,9,20,"['store', 'binari', 'data', 'mysql?']","['binari', 'data', 'mysql']","['mysql', 'database', 'binary-data', 'data-sto...",4,"['binari', 'data', 'mysql', 'store', 'binari',...",binari data mysql store binari data mysql?,mysql database binary-data data-storage
8,I'm looking for the fastest way to obtain the ...,What is the fastest way to get the value of π?,<performance><algorithm><language-agnostic><un...,23,80,"[""i'm"", 'look', 'fastest', 'way', 'obtain', 'v...","['fastest', 'way', 'get', 'valu', 'π?']","['performance', 'algorithm', 'language-agnosti...",5,"['fastest_way', 'get', 'valu', 'π?', ""i'm"", 'l...",fastest_way get valu π? i'm look fastest_way o...,performance algorithm language-agnostic unix pi
9,If I have a trigger before the update on a tab...,Throw an error in a MySQL trigger,<mysql><database><triggers>,6,25,"['trigger', 'updat', 'table,', 'throw', 'error...","['throw', 'error', 'mysql', 'trigger']","['mysql', 'd

In [17]:
X_C = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//X_C.npz')
X_T = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//X_T.npz')
Y_C = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//Y_C.npz')
Y_T = scipy.sparse.load_npz('C:/Users/ayoub/OCP6 Data//Y_T.npz')

In [14]:
with open("C:/Users/ayoub/OCP6 Data/X_C_columns.txt", "rb") as fp:
    X_C_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/X_T_columns.txt", "rb") as fp:
    X_T_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/Y_C_columns.txt", "rb") as fp:
    Y_C_columns = pickle.load(fp)
with open("C:/Users/ayoub/OCP6 Data/Y_T_columns.txt", "rb") as fp:
    Y_T_columns = pickle.load(fp)

In [21]:
X_C.shape, X_T.shape, Y_C.shape, Y_T.shape

((103676, 31541), (103676, 31541), (103676, 2639), (103676, 2639))

In [22]:
pd.DataFrame(Y_T.toarray(), columns = Y_T_columns).head()

,.htaccess,.net,.net-1.1,.net-2.0,.net-3.0,.net-3.5,.net-4.0,2d,32-bit,32bit-64bit,3d,64-bit,abap,absolute-path,abstract,abstract-class,abstract-syntax-tree,abstraction,acceptance-testing,access-control,access-modifiers,access-vba,access-violation,accessibility,accessor,accordion,acl,action,actionlink,actionresult,actionscript,actionscript-2,actionscript-3,active-directory,activemq,activerecord,activeresource,activex,adam,add-in,add-on,admin,administration,ado,ado.net,adobe,ads,adsi,aes,aggregate,aggregate-functions,aggregation,agile,air,aix,ajax,ajaxcontroltoolkit,alerts,algebra,algorithm,alias,alignment,allocation,alphablending,alter-table,amazon,amazon-ec2,amazon-s3,amazon-simpledb,amazon-web-services,amf,analysis,analytics,anchor,android,android-activity,angle,animated-gif,animation,ankhsvn,annotations,anonymous-class,anonymous-methods,anonymous-types,ant,anti-patterns,antialiasing,antivirus,antlr,aop,apache,apache-commons,apache-flex,apache-poi,apache2,apc,api,api-design,app-config,app-store,...,windbg,window,windows,windows-7,windows-authentication,windows-ce,windows-explorer,windows-forms-designer,windows-installer,windows-mobile,windows-server-2003,windows-server-2008,windows-services,windows-shell,windows-vista,windows-xp,winforms,winsock,wireless,wix,wix3,wizard,wmd,wmi,wndproc,wolfram-mathematica,word-vba,word-wrap,wordpress,workflow,workflow-foundation,worksheet-function,workspace,wpf,wpf-controls,wpftoolkit,wrapper,ws-security,wsdl,wse,wsgi,wsh,wss,wss-3.0,wxpython,wxwidgets,wysiwyg,x11,x509,x86,x86-64,xaml,xampp,xbap,xbox360,xcode,xdebug,xhtml,xib,xls,xml,xml-attribute,xml-comments,xml-documentation,xml-namespaces,xml-rpc,xml-serialization,xmldocument,xmlhttprequest,xmlreader,xmpp,xna,xor,xpath,xps,xquery,xsd,xsl-fo,xslt,xslt-1.0,xslt-2.0,xss,xstream,xul,xulrunner,yacc,yahoo,yahoo-pipes,yaml,yield,youtube,yui,z-index,zend-db-table,zend-form,zend-framework,zip,zlib,zoom,zsh
0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.427797,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0

2020-02-02 13:36:12,961 : INFO : 1:36: E251 unexpected spaces around keyword / parameter equals
2020-02-02 13:36:12,962 : INFO : 1:38: E251 unexpected spaces around keyword / parameter equals
2020-02-02 13:36:12,963 : INFO : 1:51: W291 trailing whitespace


## Unsupervised approach

### Building the Topic Model with LDA 

In [62]:
Cvectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), max_df=0.5, min_df=10)
X = Cvectorizer.fit_transform(df.sentence)

2020-02-02 16:29:09,161 : INFO : 1:80: E501 line too long (86 > 79 characters)
2020-02-02 16:29:09,167 : INFO : 1:80: E501 line too long (86 > 79 characters)


In [63]:
lda_tf = LatentDirichletAllocation(n_components=30, random_state=0)
lda_tf.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=30, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [65]:
data = pyLDAvis.sklearn.prepare(lda_tf, X, Cvectorizer, mds='mmds')

C:\Users\ayoub\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [68]:
pyLDAvis.show(data)

127.0.0.1 - - [02/Feb/2020 17:00:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2020 17:00:02] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2020 17:00:02] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2020 17:00:02] "GET /LDAvis.js HTTP/1.1" 200 -


After playing with pyLDavis, i would say that it will be complicated to do all the work and predict tags with an unsupervised approch, so I will try some classification methods

## Supervised approach

# Brouillon

In [59]:
Cvectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), max_df=0.5, min_df=10)
X = Cvectorizer.fit_transform(df_test.sentence)

2020-02-02 16:28:22,306 : INFO : 1:80: E501 line too long (86 > 79 characters)
2020-02-02 16:28:22,315 : INFO : 1:80: E501 line too long (86 > 79 characters)


In [60]:
lda_tf = LatentDirichletAllocation(n_components=30, random_state=0)
lda_tf.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=30, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [61]:
pyLDAvis.sklearn.prepare(lda_tf, X, Cvectorizer, mds='mmds')

C:\Users\ayoub\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5      0.125705  0.125638       1        1  8.455237
6      0.122868 -0.061789       2        1  6.999716
2     -0.292575 -0.069649       3        1  6.538027
0     -0.193249  0.002884       4        1  6.129671
19    -0.076230 -0.025803       5        1  4.574845
29     0.319459  0.045576       6        1  4.423067
22     0.024162  0.301838       7        1  4.381350
26    -0.062595 -0.186492       8        1  4.260171
9      0.188313  0.311217       9        1  4.119474
13     0.211905  0.248336      10        1  4.020275
11     0.308231 -0.086084      11        1  3.970912
21    -0.022864  0.166999      12        1  3.931533
24    -0.219020 -0.176363      13        1  3.740431
28    -0.107507  0.401742      14        1  3.290887
27     0.007672 -0.339615      15        1  3.264363
20    -0.273797  0.127695      16        1  3.219199
3     -0.459961 -0.100988      17        1  3.049360
25    -0.369703  0.215365      18        1  2.861408
8     -0.167497  0.215415      19        1  2.690374
12     0.356488  0.159258      20        1  2.563499
1     -0.143527  0.118690      21        1  2.327433
23     0.268165 -0.380470      22        1  2.248955
15     0.249975  0.111790      23        1  2.201943
10    -0.066486 -0.397660      24        1  1.912195
14    -0.236362 -0.300520      25        1  1.799361
7      0.387836 -0.216109      26        1  1.428343
18     0.067179 -0.229125      27        1  0.755886
4      0.017806  0.006074      28        1  0.280694
16     0.017806  0.006074      29        1  0.280694
17     0.017806  0.006074      30        1  0.280694, topic_info=   Category       Freq       Term      Total  loglift  logprob
1   Default  51.000000          =  51.000000  30.0000  30.0000
27  Default  25.000000        one  25.000000  29.0000  29.0000
37  Default  33.000000       user  33.000000  28.0000  28.0000
39  Default  32.000000       want  32.000000  27.0000  27.0000
7   Default  28.000000     applic  28.000000  26.0000  26.0000
11  Default  31.000000    databas  31.000000  25.0000  25.0000
40  Default  27.000000        way  27.000000  24.0000  24.0000
36  Default  24.000000        tri  24.000000  23.0000  23.0000
19  Default  37.000000        i'm  37.000000  22.0000  22.0000
9   Default  22.000000      creat  22.000000  21.0000  21.0000
29  Default  26.000000        run  26.000000  20.0000  20.0000
26  Default  25.000000        new  25.000000  19.0000  19.0000
38  Default  24.000000    version  24.000000  18.0000  18.0000
15  Default  17.000000      gener  17.000000  17.0000  17.0000
16  Default  37.000000        get  37.000000  16.0000  16.0000
25  Default  24.000000       need  24.000000  15.0000  15.0000
42  Default  30.000000       work  30.000000  14.0000  14.0000
10  Default  17.000000       data  17.000000  13.0000  13.0000
12  Default  21.000000       file  21.000000  12.0000  12.0000
34  Default  14.000000      start  14.000000  11.0000  11.0000
43  Default  42.000000      would  42.000000  10.0000  10.0000
33  Default  20.000000     someth  20.000000   9.0000   9.0000
2   Default  11.000000        abl  11.000000   8.0000   8.0000
8   Default  17.000000       code  17.000000   7.0000   7.0000
20  Default  21.000000        i'v  21.000000   6.0000   6.0000
41  Default  11.000000        web  11.000000   5.0000   5.0000
28  Default  13.000000    problem  13.000000   4.0000   4.0000
31  Default  24.000000        set  24.000000   3.0000   3.0000
14  Default  23.000000   function  23.000000   2.0000   2.0000
6   Default  12.000000      anyon  12.000000   1.0000   1.0000
38   Topic1  10.183315    version  24.815134   1.5797  -2.0695
4    Topic1   3.880366       also  11.467673   1.3868  -3.0344
19   Topic1  10.459868        i'm  37.253404   1.2002  -2.0427
20   Topic1   5.731664        i'v  21.178350   1.1634  -2.6443
2    Topic1   2.918292        abl  11.450824   1.1033  -